## 1. Example of Minmax Featurization


In [177]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin ## inheritance..
class Minmaxreg(BaseEstimator,TransformerMixin):
    def __init__(self,a) -> None:
        self.a=a
    def fit(self,X):
        self.X_max=np.max(X)
        self.X_min=np.min(X)
        return self
    def transform(self,X):
        
        try:
            getattr(self, "X_min") or getattr(self,"X_max")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")
        X=X.copy()
        X=(X-self.X_min)/(self.X_max-self.X_min)
        return X
    

## Output


In [178]:
X_train=[1,2,3,4,5,6,7,8,9,10]
X_test=[1,3,5]
temp=4
scale=Minmaxreg(temp)
scale.fit(X_train)
print("X_train_reg: {}".format(scale.transform(X_train))) ## 결과 1

scale.fit(X_test)
print("X_test_reg: {}".format(scale.transform(X_test))) ## 결과 2

X_train_reg: [0.         0.11111111 0.22222222 0.33333333 0.44444444 0.55555556
 0.66666667 0.77777778 0.88888889 1.        ]
X_test_reg: [0.  0.5 1. ]


In [179]:

print("X_train_reg: {}".format(scale.fit_transform(X_train))) ## 결과 1
print("X_test_reg: {}".format(scale.transform(X_test))) ## 결과 2

X_train_reg: [0.         0.11111111 0.22222222 0.33333333 0.44444444 0.55555556
 0.66666667 0.77777778 0.88888889 1.        ]
X_test_reg: [0.         0.22222222 0.44444444]


In [180]:
scale.get_params()
scale.set_params(**{'a':1653245})

Minmaxreg(a=1653245)

## 2. Ridge Rigression class

아래코드는 본강의 의 HW-2 에있는 ridge_regression.py를 그대로 작성한 것입니다.

In [187]:
from sklearn.base import BaseEstimator, RegressorMixin
from scipy.optimize import minimize, leastsq
from setup_problem import *
import numpy as np
import pandas as pd
from gradient import *
class RidgeRegression(BaseEstimator, RegressorMixin):
    """ ridge regression"""

    def __init__(self, l2reg=1):
        if l2reg < 0:
            raise ValueError('Regularization penalty should be at least 0.')
        self.l2reg = l2reg
        self.w_=0

    def fit(self, X, y=None):
        n, num_ftrs = X.shape
        # convert y to 1-dim array, in case we're given a column vector
        y = y.reshape(-1)
        def ridge_obj(w):
            predictions = np.dot(X,w)
            residual = y - predictions
            empirical_risk = np.sum(residual**2) / n
            l2_norm_squared = np.sum(w**2)
            objective = empirical_risk + self.l2reg * l2_norm_squared
            return objective
        self.ridge_obj_ = ridge_obj

        w_0 = np.random.rand(num_ftrs)
        self.w_ = minimize(ridge_obj,w_0).x
        return self

    def predict(self, X, y=None):
        try:
            getattr(self, "w_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")
        return np.dot(X, self.w_)

    def score(self, X, y):
        # Average square error
        try:
            getattr(self, "w_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")
        y = y.reshape(-1)
        residuals = self.predict(X) - y
        return np.dot(residuals, residuals)/len(y)
    

In [188]:
index=np.arange(0,100,1)
N = 100
theta = np.array([[1], [3]])
X = np.c_[np.random.rand(N,1), np.ones((N,1))]
y = np.dot(X,theta)+ 0.4*np.random.randn(N,1)
ridge=RidgeRegression(l2reg=0.01)
ridge.fit(X,y)
print(ridge.predict(X))
print(ridge.score(X,y))

[3.88611996 3.56771314 3.75686296 3.45612732 3.33523948 3.4524712
 3.25370604 3.25374748 3.85940386 3.12527637 3.14969656 3.46280067
 3.66479507 3.24839118 3.29621433 3.08322493 3.8973187  3.87058627
 3.28568792 3.86812922 3.68765039 3.94750866 3.65141152 3.52469342
 3.82699152 3.85078643 3.20908756 3.51396306 3.61286517 3.04863113
 3.20328745 3.8458258  2.96595707 3.32467843 3.17910169 3.76505065
 3.03738217 3.8650506  3.18467165 3.82363615 3.26044198 3.68788051
 3.9382866  3.21256968 3.97520851 3.45123161 3.61641466 3.54726602
 3.90809074 3.2077914  3.68273243 3.14675312 3.27072461 3.31915187
 3.53459214 3.25024768 3.94204119 3.51524523 3.89919966 2.96100786
 3.46022365 3.46334131 3.46552773 3.88932486 3.05532607 3.8382798
 3.36692323 3.44178414 3.06620669 3.82367069 3.80888024 3.49907278
 3.1686451  3.68592752 3.41158578 3.15324427 2.95972733 3.68099365
 3.59133152 3.17308082 3.88546797 3.97002535 3.26510667 3.80953833
 2.94736561 3.45244444 3.92657169 3.0346473  3.47281658 3.915236